In [13]:
import os
from os import path
import re
import urllib
import urllib.request
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup as soup
import requests
from bs4 import BeautifulSoup, SoupStrainer
from google.cloud import speech_v1p1beta1 as speech
import speech_recognition as sr
from pydub import AudioSegment
import subprocess
import json
import requests

In [14]:
def soup_maker(link: str):
    """ takes in a html link as a string and returns a page soup """
    req_single = Request(link , headers={'User-Agent': 'Mozilla/5.0'})
    single_page = urlopen(req_single).read()
    return BeautifulSoup(single_page, "html.parser")

def mp3_scrape(page_soup):
    """ takes in a page soup and returns one mp3 link of that episode """
    text = page_soup.prettify()
    text = text.replace("\\/", "/").replace(" ","")
    
    two_players = re.findall(r'SmartPodcastPlayer.*?=.*?}}',text)
    curr_player = two_players[0]
    json_text = '{'+curr_player.split('={')[1]
    json_data = json.loads(json_text)
    mp3_url = json_data['options']['url']
    print(mp3_url)

    # regex = re.findall(r'http.*?mp3', text)
    # sum = ''
    # for element in regex:
    #     sum += element
    # res = sum.split(',')
    # 
    # links = [re.findall(r'http.*?mp3', i) for i in res]
    # unpacked = [i[0] for i in links if i != []]
    # unique_list = list(set(unpacked))

    return mp3_url

def download_mp3(folder: str, link: str):
    if not os.path.exists(folder):
        os.makedirs(folder)

    filename = link.split('/')[-1]
    url = f'{folder}{filename}'
    urlretrieve(link, url)
        
    # filenames = []
    # for url in unique_list:
    #     filenames.append(url.split('/')[-1])
    # 
    # for ind, ele in enumerate(unique_list):
    #     url = f'{folder}{filenames[ind]}'
    #     urlretrieve(ele, url)

    return filename


# Convert MP3 to WAV
def convert_to_wav_sox(mp3_file):
    # assign files 
    input_file = mp3_file
    output_file = input_file[:-4]+'.wav'

    # convert mp3 file to wav file 
    sound = AudioSegment.from_mp3(input_file)
    sound.export(output_file, format="wav")
    return output_file

# Text transcribe
def transcribe_audio(audio_file, duration):
    recognizer = sr.Recognizer()

    # Load audio file
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source, duration)

    # Perform the transcription using CMU Sphinx
    try:
        transcription = recognizer.recognize_sphinx(audio_data)
        return transcription
    except sr.UnknownValueError:
        return "Sphinx could not understand the audio"
    except sr.RequestError as e:
        return f"Error occurred in Sphinx; {e}"

In [15]:
# Extract data from json
filename = 'config.json'
relative_path = os.path.join('..', filename)
absolute_path = os.path.abspath(relative_path)

# Check if the file exists
if os.path.exists(absolute_path):
    # Load JSON data from the file
    with open(absolute_path, 'r') as file:
        json_data = json.load(file)
    # Access the loaded JSON data
    print("JSON data loaded from the file in the parent directory:")
else:
    print("File does not exist in the parent directory.")

# Take first URL of firs
archive_url = json_data['podcastTargets'][0]['urls'][0]
print(archive_url)

JSON data loaded from the file in the parent directory:
https://peterattiamd.com/podcast/archive/


In [16]:
# Scape archive site with all the links to podcasts
page_soup = soup_maker(archive_url)

article = page_soup.find('article')
hrefs = [a['href'] for a in article.findAll('a', href=True)]
print(hrefs[:5])

['https://peterattiamd.com/daxshepard/', 'https://peterattiamd.com/lucvanloon/', 'https://peterattiamd.com/paulconti4/', 'https://peterattiamd.com/ama58/', 'https://peterattiamd.com/courtneyconley/']


In [17]:
# downloading all mp3 links into audio folder
folder = "audios/"
filenames = []
final_texts = []

duration = 20
hrefs = hrefs[:3]
for link in hrefs:
    soup = soup_maker(link)
    mp3_link = mp3_scrape(soup)
    filename = download_mp3(folder,mp3_link)
    # filenames = filenames + download_mp3(folder, mp3_links)
    
    audio_file_path = folder+filename
    wav_file_path = convert_to_wav_sox(audio_file_path)
    transcription = transcribe_audio(wav_file_path, duration)

    data = {
        "filename": filename,
        "transcription": transcription
    }

    # Append dictionary to list
    final_texts.append(data)

# Define the output JSON file path
output_file = "podcast_transcriptions.json"

# Save list of dictionaries as JSON to a file
with open(output_file, "w") as json_file:
    json.dump(final_texts, json_file, indent=4)

# Print JSON string
print(f"Transcriptions saved to {output_file}")


    
# # go through mp3's and convert and transcribe them  
# for file in filenames:
#     convert_and_transcribe(folder, file, duration)


https://traffic.libsyn.com/secure/peterattiadrive/EP.XX_-_Dax_Shepard_on_Senna_FINAL.mp3
https://traffic.libsyn.com/secure/peterattiadrive/EP.299_-_Luc_van_Loon_PUBLIC.mp3
https://traffic.libsyn.com/secure/peterattiadrive/EP.298_-_Paul_Conti_4_FINAL.mp3
Transcriptions saved to podcast_transcriptions.json


## Achtung ... Baustelle!

In [18]:
r = sr.Recognizer()

audio_file = sr.AudioFile('audios/'+'EP.254_-_AMA_47_SNEEK_PEEK.wav')

with audio_file as source:
    audio = r.record(source, duration=40)

"""    
with audio_file as source:
    audio = r.record(source)
"""
# text = r.recognize_google(audio)

FileNotFoundError: [Errno 2] No such file or directory: 'audios/EP.254_-_AMA_47_SNEEK_PEEK.wav'

In [ ]:
# Function to transcribe audio to text using Google Cloud Speech-to-Text
def transcribe_audio(audio_url):
    client = speech.SpeechClient()
    audio = speech.RecognitionAudio(uri=audio_url)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.OGG_OPUS,
        language_code="en-US",
    )
    response = client.recognize(config=config, audio=audio)
    transcripts = []
    for result in response.results:
        transcripts.append(result.alternatives[0].transcript)
    return transcripts

# URL of the webpage containing the audio
url = "https://content.libsyn.com/p/d/1/b/d1b65ad733f9441c/EP.254_-_AMA_47_SNEEK_PEEK.mp3?c_id=151844049&cs_id=151844049&response-content-type=audio%2Fmpeg&Expires=1714302958&Signature=ZoU1VHP2md8aJSWHfUdjpLK3~iZ1II0G7hhFxN0wPC50NGUw7jcNL3O4zwbJsgzeQdWcakT2rsAe-qO-VJqXD1jkDX9glPox~inBEJibxcwlH7jhb3fyPITfa8BnYBlmkl~6yiI6oT1VejMRWV1SGt9ed-YvAhMW3japlpRd2cBTGB3ioxDuDE7JWB5pJ6YvF7hmTd6oq2a94rycanbTI5kpBwcb3O6eFl5XI4iY0hj-c9bfjbFeangw5J7ZYaGh~B5BvdSRvN0EQVcLf5KTy4tF~LI8a0FMKcqf-I5BFkLGUcACwJnINNkgFBefmZ3LvHyig2V4lwH4FDHdY-IOdg__&Key-Pair-Id=K1YS7LZGUP96OI" 
# "https:\/\/traffic.libsyn.com\/secure\/peterattiadrive\/EP.254_-_AMA_47_SNEEK_PEEK.mp3"
# "https://content.libsyn.com/p/d/1/b/d1b65ad733f9441c/EP.254_-_AMA_47_SNEEK_PEEK.mp3?c_id=151844049&cs_id=151844049&response-content-type=audio%2Fmpeg&Expires=1714302958&Signature=ZoU1VHP2md8aJSWHfUdjpLK3~iZ1II0G7hhFxN0wPC50NGUw7jcNL3O4zwbJsgzeQdWcakT2rsAe-qO-VJqXD1jkDX9glPox~inBEJibxcwlH7jhb3fyPITfa8BnYBlmkl~6yiI6oT1VejMRWV1SGt9ed-YvAhMW3japlpRd2cBTGB3ioxDuDE7JWB5pJ6YvF7hmTd6oq2a94rycanbTI5kpBwcb3O6eFl5XI4iY0hj-c9bfjbFeangw5J7ZYaGh~B5BvdSRvN0EQVcLf5KTy4tF~LI8a0FMKcqf-I5BFkLGUcACwJnINNkgFBefmZ3LvHyig2V4lwH4FDHdY-IOdg__&Key-Pair-Id=K1YS7LZGUP96OI" 
#"https://peterattiamd.com/ama47/"

# Scrape the audio URL
audio_url = scrape_audio_url(url)
if audio_url:
    print("Audio URL found:", audio_url)
    # Transcribe the audio to text
    # transcripts = transcribe_audio(audio_url)
    print("Transcription:")
    for transcript in transcripts:
        print(transcript)
else:
    print("No audio found on the webpage.")

In [ ]:
test_mp3 = filenames[0]

client = speech.SpeechClient()

# Configure audio settings
config = {
    "language_code": "en-US",
}
audio = {"uri": test_mp3}

# Perform the transcription
response = client.long_running_recognize(config=config, audio=audio)
operation = response.operation
operation_result = operation.result()

# Parse the transcription results
transcription = ""
for result in operation_result.results:
    for alternative in result.alternatives:
        transcription += alternative.transcript + "\n"

# Provide the path to your audio file
audio_file_path_1 = test_mp3

# Call the transcribe_audio function
# transcription_1 = transcribe_audio(audio_file_path_1)
print("Transcription:")
# print(transcription_1)

In [ ]:
r = sr.Recognizer()

audio_file = sr.AudioFile(folder+'EP.254_-_AMA_47_SNEEK_PEEK.wav')

with audio_file as source:
    audio = r.record(source, duration=40)

"""    
with audio_file as source:
    audio = r.record(source)
"""
# text = r.recognize_google(audio)
# print(text)